In [1]:
!pip install fastai

     |████████████████████████████████| 194 kB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 4.5 MB/s  eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [2]:
from fastai.vision.all import *

In [5]:
import pandas as pd

In [3]:
datablocks = DataBlock()

In [6]:
df = pd.read_csv('train.csv')

In [7]:
len(df)

800000

In [8]:
y = df['label']
x = df.fillna(0)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x,y,
                                                    test_size=0.1,
                                                    random_state=42,
                                                    stratify=y)

In [15]:
y = x_test['label']
x = x_test.fillna(0)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x,y,
                                                    test_size=0.1,
                                                    random_state=42,
                                                    stratify=y)

In [17]:
len(x_test)

8000

In [18]:
x_test.to_csv('subset.csv')

In [39]:
embedding=list(x_test.caption_embedding)

In [22]:
value=embedding[0].split(',')

In [29]:
value = [float(i) for i in value]

In [30]:
type(value[0])

float

In [38]:
def get_y(caption_embedding):
    value=caption_embedding.split(',')
    value.pop(0)
    embedding_val = [float(i) for i in value]
    return embedding_val

In [42]:
x_test.head()

,caption,img,label,caption_embedding
484617,captions_resized_1M/cities_instagram/singapore/1482035932717737096.txt,img_resized_1M/cities_instagram/singapore/1482035932717737096.jpg,singapore,",0.45350155971180706,0.3800610459612754,0.3849763393052144,0.4705067313420381,0.7200443415374759,0.9018688458171419,0.4514048428742117,0.43210801174895436,0.686250401161629,0.47221142328592264,0.630813467100926,0.895093776698023,0.6617422366222705,0.23865608963107246,0.3940583823045318,0.5626082162489685,0.40785245677608467,0.8340910643014352,0.7785953289859912,0.3822970511166835,0.6582121043157304,0.5636783254503643,0.4314199464323862,0.2964874136299024,0.6410295859583822,0.6108022948250351,0.5292383822619953,0.37298645503169653,0.5864829065402798,0.6911801236914589,0.2987429101249909,0.9..."
351530,captions_resized_1M/cities_instagram/toronto/1489407011283035254.txt,img_resized_1M/cities_instagram/toronto/1489407011283035254.jpg,toronto,",0.45350155971180706,0.3800610459612754,0.3849763393052144,0.4705067313420381,0.7200443415374759,0.9018688458171419,0.4514048428742117,0.43210801174895436,0.686250401161629,0.47221142328592264,0.630813467100926,0.895093776698023,0.6617422366222705,0.23865608963107246,0.3940583823045318,0.5626082162489685,0.40785245677608467,0.8340910643014352,0.7785953289859912,0.3822970511166835,0.6582121043157304,0.5636783254503643,0.4314199464323862,0.2964874136299024,0.6410295859583822,0.6108022948250351,0.5292383822619953,0.37298645503169653,0.5864829065402798,0.6911801236914589,0.2987429101249909,0.9..."
656953,captions_resized_1M/cities_instagram/sanfrancisco/1482261340008562671.txt,img_resized_1M/cities_instagram/sanfrancisco/1482261340008562671.jpg,sanfrancisco,",0.45350155971180706,0.3800610459612754,0.3849763393052144,0.4705067313420381,0.7200443415374759,0.9018688458171419,0.4514048428742117,0.43210801174895436,0.686250401161629,0.47221142328592264,0.630813467100926,0.895093776698023,0.6617422366222705,0.23865608963107246,0.3940583823045318,0.5626082162489685,0.40785245677608467,0.8340910643014352,0.7785953289859912,0.3822970511166835,0.6582121043157304,0.5636783254503643,0.4314199464323862,0.2964874136299024,0.6410295859583822,0.6108022948250351,0.5292383822619953,0.37298645503169653,0.5864829065402798,0.6911801236914589,0.2987429101249909,0.9..."
705342,captions_resized_1M/cities_instagram/losangeles/1490064519439121400.txt,img_resized_1M/cities_instagram/losangeles/1490064519439121400.jpg,losangeles,",0.45350155971180706,0.3800610459612754,0.3849763393052144,0.4705067313420381,0.7200443415374759,0.9018688458171419,0.4514048428742117,0.43210801174895436,0.686250401161629,0.47221142328592264,0.630813467100926,0.895093776698023,0.6617422366222705,0.23865608963107246,0.3940583823045318,0.5626082162489685,0.40785245677608467,0.8340910643014352,0.7785953289859912,0.3822970511166835,0.6582121043157304,0.5636783254503643,0.4314199464323862,0.2964874136299024,0.6410295859583822,0.6108022948250351,0.5292383822619953,0.37298645503169653,0.5864829065402798,0.6911801236914589,0.2987429101249909,0.9..."
535400,captions_resized_1M/cities_instagram/miami/1490034567225075835.txt,img_resized_1M/cities_instagram/miami/1490034567225075835.jpg,miami,",0.45350155971180706,0.3800610459612754,0.3849763393052144,0.4705067313420381,0.7200443415374759,0.9018688458171419,0.4514048428742117,0.43210801174895436,0.686250401161629,0.47221142328592264,0.630813467100926,0.895093776698023,0.6617422366222705,0.23865608963107246,0.3940583823045318,0.5626082162489685,0.40785245677608467,0.8340910643014352,0.7785953289859912,0.3822970511166835,0.6582121043157304,0.5636783254503643,0.4314199464323862,0.2964874136299024,0.6410295859583822,0.6108022948250351,0.5292383822619953,0.37298645503169653,0.5864829065402798,0.6911801236914589,0.2987429101249909,0.9..."
